# bag-of-words tekstklassifikasjon med Keras

Det er lenge side bag-of-words var avantgarde, men metoden er enkel og fungerer fortsatt veldig godt på mange typer tekst. Det er blant annet dette som brukes av KPI når de klassifiserer varer. Mens KPI bruker R og SVM, bruker vi anledningen til å fortsette med Keras.

I dette eksempelet forsøker vi en binær klassifisering av dokumenttitler fra oep.no (nå einnsyn.no), for å forsøke å skille dokumenttitler fra SSB og dokumenttitler fra fylkesmannen i Sogn og Fjordane. Gruppene er nesten nøyaktig like store og vi har derfor en baseline på 50% accuracy. Vi forventer derimot en ganske høy accuracy på dette problemet, fordi SSB og fylkesmannen driver med veldig forskjellige ting, i tillegg til at fylkesmannen i Sogn og Fjordane skriver nynorsk. Vi har med andre ord et veldig sterkt signal å gå etter.

In [1]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers.core import Dense, Activation

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import html
from nltk.corpus import stopwords
import re

Using TensorFlow backend.


In [2]:
OEPFILE = '/data/kurs/python/ssb_fylkesmann_sogf.parquet'

In [3]:
df = (pd.read_parquet(OEPFILE)
      .query("agency=='SSB' | agency=='FMSF'") # Filtrerer noen feilrecordinger
      .apply(lambda x: html.unescape(x)) # Oversetter fra HTML-escapetegn
     )

In [4]:
len(df)

127781

In [5]:
df.head()

,id,bid,bureau,case_title,doc_title,case_n,doc_n,agency,journal_date,doc_date,pub_date,to_from,excempt,archive_code
0,89048,188,Statistisk sentralbyrå,M&#229;nedlig unders&#248;kelse for overnattin...,Nye filer med ilagt tvangsmulkt Overnattinger ...,2014/99,199,SSB,01.12.2014,01.12.2014,04.12.2014,TO: Statens innkrevingssentral,None,None
1,89049,188,Statistisk sentralbyrå,Rapportering til Statsregnskapet for 2014,Statsregnskapet for 2014 - &#229;rsavslutning ...,2014/1452,1,SSB,01.12.2014,01.12.2014,04.12.2014,FROM: Finansdepartementet,None,None
2,89050,188,Statistisk sentralbyrå,Personalmappe,Forlengelse av engasjement,2011/1243,17,SSB,02.12.2014,01.12.2014,05.12.2014,Internal,"Offentleglova &#167; 25, f&#248;rste ledd.",None
3,89051,188,Statistisk sentralbyrå,Godstransport med norske lastebiler 2014 - Uts...,Purring/varsel om tvangsmulkt - Godstransport ...,2013/1621,129,SSB,02.12.2014,01.12.2014,05.12.2014,TO: Oppgavegivere i f&#248;lge brev,Offentleglova &#167; 13 jf. Statistikkloven &#...,None
4,89052,188,Statistisk sentralbyrå,H&#248;ring - Overf&#248;ring av skatteoppkrev...,H&#248;ring med svarfrist 020315 - Overf&#248;...,2014/1458,1,SSB,02.12.2014,01.12.2014,05.12.2014,FROM: Finansdepartementet,None,None


In [6]:
df['agency'].value_counts()

SSB     63962
FMSF    63819
Name: agency, dtype: int64

Fordi train_test_split fungerer påfallende dårlig gjør vi et annet triks: Sampling 100% uten tilbakelegging gjør i praksis det samme som å sortere datasettet i tilfeldig rekkefølge. Dette gjør at vi kan bruke de N første radene til trening, og resten til test.

In [7]:
df = df.sample(frac=1)

## Preprosessere y-variabelen

Vi skal forsøke å predikere `agency`-variabelen, men for å gjøre det må vi konvertere den fra tekst til tall. Den enkleste måten å gjøre dette på er å regne ut en ny kolonne som er 0/1, men denne løsningen generaliseres ikke til flerklasseproblem. Derfor bruker vi noen innebygde funksjoner fra keras og sklearn som gjør dette for oss på en litt mer generell måte.

In [8]:
from keras.preprocessing.text import one_hot

In [9]:
y = df['agency']
labelencoder = LabelEncoder()
y_encoded = labelencoder.fit_transform(y)

num_classes = np.max(y_encoded) + 1
yhot = to_categorical(y_encoded, num_classes)

In [10]:
yhot.shape

(127781, 2)

Det vi nå sitter igjen med er en matrise som er to bred, selv om den egentlig bare hadde trengt å være 1. For å bruke regresjonsnomenklaturen har ikke matrisen en base-verdi. Keras fungerer utmerket med dette, så lenge vi tar høyde for det når vi lager modellen vår.

## Preprossesere tekst

Preprosessering av tekst er en langt mer omstendig prosess, som er både en vitenskap og en kunst. Prosessen kan innebære en del ulike steg, et minimum er å konvertere til små bokstaver og fjerne tegnsetting. Men ytterligere steg kan blant annet være å fjerne ord som ikke inneholder relevant informasjon (som f.eks. *og*, *å*, *til*, etc). I nomenklaturen heter dette stoppord, og norsk har ca 140-160 stoppord avhengig av hvem du spør. I tillegg gjøres ofte det som heter "stemming" som går ut på å fjerne endelser, bøyinger og annet som avhenger av gramatikk. Her gjør vi det dog ganske enkelt.

In [11]:
import codecs
stops = codecs.open('data/stoppord.txt', 'r', encoding='utf8').readlines()

def alphanum(text):
    return re.sub("[^A-Za-zæÆøØåÅ]", " ", text)

def remove_stops(text):
    return " ".join([word for word in text.split() if word not in stops])

def clean_text(text):
    cleantext = text.lower()
    cleantext = alphanum(cleantext)
    cleantext = remove_stops(cleantext)
    return cleantext

In [12]:
X = df['doc_title'].apply(clean_text)

In [13]:
X[:10]

12950     foresp rsel avklaringer tilbud aksept av tilbu...
59054                 s knad om ettergivelse av tvangsmulkt
64828                foresp rsel om tilbud p matlab toolbox
89526                                          s knad og cv
125056    s knad om fritak fra opplysningsplikten er avs...
64044                                   anskaffelsesrapport
38713                                           avslagsbrev
67032                                          s knad og cv
73503                                  tilleggsopplysningar
80970     tilsyn ved veidekke avd gloppen inspeksjonsrap...
Name: doc_title, dtype: object

In [14]:
tokenizer = Tokenizer(num_words = 2000)
tokenizer.fit_on_texts(X)
X_matrix = tokenizer.texts_to_matrix(X, mode='binary')

In [15]:
import math

In [16]:
train_size = math.floor(len(df)*0.8)

X_train, X_test, y_train, y_test = X_matrix[0:train_size], X_matrix[train_size:], yhot[0:train_size], yhot[train_size:]
outputlayers = yhot.shape[1]

In [17]:
# X_matrix er enormt stor, sletter for å frigjøre minne
del(X_matrix)

In [18]:
model = Sequential()
model.add(Dense(2048, input_shape = (2000,)))
model.add(Activation('relu'))
model.add(Dense(outputlayers))
model.add(Activation('softmax'))

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test),
                    batch_size=512,
                    epochs=2)

Train on 102224 samples, validate on 25557 samples
Epoch 1/2
102224/102224 [==============================] - 15s 147us/step - loss: 0.0974 - acc: 0.9666 - val_loss: 0.0559 - val_acc: 0.9774
Epoch 2/2
102224/102224 [==============================] - 10s 101us/step - loss: 0.0451 - acc: 0.9821 - val_loss: 0.0498 - val_acc: 0.9808


## Prøv selv

Om du ikke har egne ideer selv, kan du forsøke en av disse:

- Finn et nytt sett med institusjoner, og forsøk å skille disse. Bruk to statlige etater i Oslo, så merker du nok at det er litt vanskeligere å skille de fordi de bruker samme målform og mange av sakene er forholdsvis generiske personalsaker.

- Forsøk å endre modellen til å bruke flere eller færre ord, og se hvordan resultatet påvirkes

- Lag din egen liste over vanlige ord i tekstene. Trolig kan en del av disse betraktes som "stoppord", og kanskje modellen blir bedre hvis de fjernes?